# Logistic Regression

_Tham khảo thêm:_

https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html

### Introduction

`Logistic regression` được dùng cho bài toán phân loại nhị phân (`binary classification`). `Ouput` (đầu ra) của logistic regression là giá trị kiểu float nằm trong khoảng `[0,1]` và có thể được dùng để phân loại nhị phân. Ví dụ chúng ta muốn xây dựng mô hình logistic regression để xác định ảnh đầu vào có chứa con mèo hay không. Lúc này ảnh chứa con mèo có thể xác định bằng cách so sánh giá trị output với một giá trị T (thường cho T = 0.5). Nếu output > T, mô hình quyết định ảnh input có chứa con mèo. Ngược lại, ảnh input không chứa con mèo.

**_So sánh với Linear Regression_**

Giả sử có bộ dữ liệu về điểm bài kiểm tra của sinh viên, đều laf input của Linear Regression và Logistic Regression. Sự khác nhau thể hiện ở:

- `Linear Regression`: dự đoán số điểm của sinh viên trong khoảng [0,100]. Linear Regression dự đoán cho hàm liên tục. 
- `Logistic Regression`:  dự đoán liệu rằng sinh viên có đỗ hay trượt. Logistic Regression dự đoán rời rạc.

**_Type of Logistic Regression_**

- Binary(Pass/Fail)
- Multi (Cats, Dogs, Sheep)
- Ordinal(Low, Medium, High)

### Binary Logistic Regression

**_Sigmoid activation_**

Hàm `Sigmoid` được sue dụng tỏng trường hợp dự đoán theo xác suất. Hàm này ánh xạ bất kỳ một giá trị thực nào thành giá trị khác trong khoảng [0,1]. Trong ML, sử dụng `Sigmoid` để ánh xạ dự doán theo xác suất.

$$ S(z) = \frac{1}{1+e^{-z}}$$

_Note:_
- s(z)  = output between 0 and 1 (probability estimate)
- z = input to the function (your algorithm’s prediction e.g. mx + b)
- e = base of natural log

_Graph:_
![sigmoid.png](img/sigmoid.png)

In [2]:
import numpy as np

def sigmoid(z, derivative=False):
    sigm = 1. /(1. + np.exp(-z))
    if derivative:
        return sigm*(1-sigm)
    return sigm

**_Decision boundary_**

Hàm dự đoán sẽ trả về một điểm xác suất trong khoảng  [0,1]. Để ánh xạ nó tới một lớp riêng biệt (đúng/sai, mèo/chó). Chúng ta sẽ chọn một giá trị ngưỡng (threshold) và phân loại các giá trị vào lớp 1 hoặc lớp 2. Ví dụ threshold = 0.5

$$ p >= 0.5 , class = 1 $$
$$ p <= 0.5 , class = 0 $$

![logistic_regression_sigmoid_w_threshold.png](img/logistic_regression_sigmoid_w_threshold.png)


**_Making predictions_**

Sử dụng `Sigmoid` và `Decison boundary`, ta sẽ tạo hàm `prediction function` . Hàm `prediction` trả về xác suất cho sự học tạp là dương, True hoặc 'Yes'. Ta gọi class này là 1 và kí hiệu: $P(class=1)$.

$$ z = \theta^{T}x$$
$$ P(class=1) = \frac{1}{1 + e^{-z}}$$

Nếu mô hình trả về 0.4 tức là 40% cơ hội 'Pass', nếu threshold = 0.5 thì phân loại là 'Fail'.

In [3]:
def predict(features, weights):
    '''
    Return 1D array of probabilities
    that the class label == 1
    '''
    
    z = np.dot(features, weights)
    return sigmoid(z)


**_Cost function_**

Thay vì sử dụng hàm Mean Squared Error, chúng ta sử dụng hàm Cross-Entropy, hay được goị là Log Loss. Cross-Entropy được chia làm 2 phần cost functions: `y=1` và `y=0`.

$$ J(\theta) = \frac{1}{m}\sum^{m}_{i=1}Cost(h_\theta(x^{(i)}), y^{(i)})$$
$Cost(h_\theta(x), y) = -log(h_\theta(x))$   if y = 1

$Cost(h_\theta(x), y) = -log(1 - h_\theta(x))$    if y = 0

![y1andy2_logistic_function.png](img/y1andy2_logistic_function.png)

**Hàm cost cụ thể**
![logistic_cost_function_joined.png](img/logistic_cost_function_joined.png)

**Vectorized cost function**
![logistic_cost_function_joined.png](img/logistic_cost_function_vectorized.png)

Code:


In [4]:
def cost_function(features, labels, weights):
    '''
    Using Mean Absolute Error
    
    Returns 1D matrix of predictions
    Cost = (labels*log(predictions) + (1-labels)*log(1-predictions) ) / len(labels)
    '''
    observations = len(labels)
    
    predictions = predict(features, weights)
    
    #take the error when label=1
    class1_cost = -labels*np.log(predictions)
    
    #take the error when label=0
    class0_cost = (1-labels)*np.log(1-predictions)
    
    #take the sum of both costs
    cost = class1_cost + class0_cost
    
    #take the average cost
    cost = cost.sum()/observations
    
    return cost

**_Gradien Descent_**

Gradient Descent được dùng để minimize giá trị hàm cost.

$$ s'(z) = s(z)(1-s(z))$$
$$ C' = x(s(z) - y) $$

_Note_

- C′ is the derivative of cost with respect to weights
- y is the actual class label (0 or 1)
- s(z) is your model’s prediction
- x is your feature or feature vector.

Repeat {

    1. Calculate Gradient average
    2. Multiply by learning rate
    3. Subtract from weights
}

Code

In [5]:
def update_weights(features, labels, weights, lr):
    '''
    Vectorized Gradient Descent
    '''
    N = len(features)
    
    #1. Get Predictions
    predictions = predict(features, weights)
    
    #2. Tranpose features so we can multiply w cost matrix.
    gradient = np.dot(features.T, predictions-labels)
    
    #3. Take the average cost derivative for each feature
    gradient /= N
    
    #4. Multiplt the gradient by our learning rate
    gradien *=lr
    
    #5. Subtract from our weights to minimize cost
    weights -= gradient
    
    return weights


**_Mapping probabilities to classes_**

Bước cuối cùng là phân nhã cho dự đoán xác suất tính được

- **Decision boundary**

In [6]:
def decision_boundary(prob):
    return 1 if prob >= 0.5 else 0


- **Covert probabilities to classes**

In [7]:
def classify(predictions):
    
    '''
    input = N element array of predictions between 0 and 1
    output = N element array of 0s(False) and 1s(True)
    '''
    
    decision_boundary = np.vectorize(decision_boundary)
    
    return decision_boundary(predictions).flatten()

**_Training_**

Code phần training tương tự với Linear Regression

In [8]:
def train(features, labels, weights, iters):
    cost_history = []
    
    for i in range(inters):
        weights = update_weights(features, labels, weights, lr)
        
        #calculate erroe for additing purposes
        cost = cost_function(features, labels, weights)
        cost_function.append(cost)
        
        #log progress
        if i % 1000 == 0:
            print("iter: "+ str(i) + "cost: " + str(cost))
            
    return weights, cost_history

### Multiclass Logistic Regression

Thay vì y = 0, 1 thì y được mở rộng ở nhiều biến y = 0, 1, ..., n

**Procedure**
    1. Coi bài toán là bài toán phân loại nhị phân n+1 (+1 vì chỉ số bắt đầu từ 0)
    2. Thực hiện với mỗi class..
    3. Dự đoán xác suất của từng bản ghi với mỗi lớp đơn
    4. Kết luận: prediction = <math>max(probability of classes)
    
**Softmax activation**

**Scikit-Learn exapmle**

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

# Normalize grades to values between 0 and 1 for more efficient computation
normalized_range = sklearn.preprocessing.MinMaxScaler(feature_range=(-1,1))

# Extract Features + Labels
labels.shape =  (100,) #scikit expects this
features = normalized_range.fit_transform(features)

# Create Test/Train
features_train,features_test,labels_train,labels_test = train_test_split(features,labels,test_size=0.4)

# Scikit Logistic Regression
scikit_log_reg = LogisticRegression()
scikit_log_reg.fit(features_train,labels_train)

#Score is Mean Accuracy
scikit_score = clf.score(features_test,labels_test)
print 'Scikit score: ', scikit_score

#Our Mean Accuracy
observations, features, labels, weights = run()
probabilities = predict(features, weights).flatten()
classifications = classifier(probabilities)
our_acc = accuracy(classifications,labels.flatten())
print 'Our score: ',our_acc